In [1]:


import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

tflib.init_tf()
with open('results/00005-sgan-custom-1gpu/network-snapshot-007024.pkl','rb') as f:
    _G, _D, Gs = pickle.load(f)

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import scipy

grid_size = [2,2]
image_shrink = 1
image_zoom = 1
duration_sec = 10.0
smoothing_sec = 1.0
mp4_fps = 20
mp4_codec = 'libx264'
mp4_bitrate = '16M'
random_seed = 404
mp4_file = 'random_grid_%s.mp4' % random_seed
minibatch_size = 8

num_frames = int(np.rint(duration_sec * mp4_fps))
random_state = np.random.RandomState(random_seed)

# Generate latent vectors
shape = [num_frames, np.prod(grid_size)] + Gs.input_shape[1:] # [frame, image, channel, component]
all_latents = random_state.randn(*shape).astype(np.float32)
all_latents = scipy.ndimage.gaussian_filter(all_latents, [smoothing_sec * mp4_fps] + [0] * len(Gs.input_shape), mode='wrap')
all_latents /= np.sqrt(np.mean(np.square(all_latents)))


def create_image_grid(images, grid_size=None):
    assert images.ndim == 3 or images.ndim == 4
    num, img_h, img_w, channels = images.shape

    if grid_size is not None:
        grid_w, grid_h = tuple(grid_size)
    else:
        grid_w = max(int(np.ceil(np.sqrt(num))), 1)
        grid_h = max((num - 1) // grid_w + 1, 1)

    grid = np.zeros([grid_h * img_h, grid_w * img_w, channels], dtype=images.dtype)
    for idx in range(num):
        x = (idx % grid_w) * img_w
        y = (idx // grid_w) * img_h
        grid[y : y + img_h, x : x + img_w] = images[idx]
    return grid

# Frame generation func for moviepy.
def make_frame(t):
    frame_idx = int(np.clip(np.round(t * mp4_fps), 0, num_frames - 1))
    latents = all_latents[frame_idx]
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.5,
                          randomize_noise=False, output_transform=fmt)

    grid = create_image_grid(images, grid_size)
    if image_zoom > 1:
        grid = scipy.ndimage.zoom(grid, [image_zoom, image_zoom, 1], order=0)
    if grid.shape[2] == 1:
        grid = grid.repeat(3, 2) # grayscale => RGB
    return grid

# Generate video.
import moviepy.editor
video_clip = moviepy.editor.VideoClip(make_frame, duration=duration_sec)
video_clip.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.03366912/45929032 bytes (7.3%7135232/45929032 bytes (15.510977280/45929032 bytes (23.9%14524416/45929032 bytes (31.6%18292736/45929032 bytes (39.8%21651456/45929032 bytes (47.1%23707648/45929032 bytes (51.6%25894912/45929032 bytes (56.4%28073984/45929032 bytes (61.1%30547968/45929032 bytes (66.5%33267712/45929032 bytes (72.4%35577856/45929032 bytes (77.5%37806080/45929032 bytes (82.3%39878656/45929032 bytes (86.8%42065920/45929032 bytes (91.6%45178880/45929032 bytes (98.4%45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1.
[MoviePy] >>>> Building video random_grid_404.mp4
[MoviePy] Writing video random_grid_404.mp4


100%|█████████▉| 200/201 [00:50<00:00,  3.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: random_grid_404.mp4 

